Install qlatent is required

In [1]:
!pip install qlatent

In [2]:
import importlib

from qlatent.qmnli.qmnli import *
from qlatent.qmnli.qmnli import _QMNLI, QMNLI
# importlib.reload(_QMNLI)

import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_p

-1


In [3]:
# softmax_files = [False, True]
# softmax_files = [True]

def split_question(Q, index, scales, softmax, filters):
  result = []
  for s in scales:
    q = QCACHE(Q(index=index, scale=s))
    for sf in softmax:
      for f in filters:
        if sf:            
            qsf = QSOFTMAX(q,dim=[index[0], s])
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print((index, s),sf,f)
            result.append(qsf_f)
            
#             qsf = QSOFTMAX(q,dim=s)
#             qsf_f = QFILTER(qsf,filters[f],filtername=f)
#             print(s,sf,f)
#             result.append(qsf_f)
            
#             qsf = QSOFTMAX(q,dim=index[0])
#             qsf_f = QFILTER(qsf,filters[f],filtername=f)
#             print(index[0],sf,f)
#             result.append(qsf_f)
        else:
            qsf = QPASS(q,descupdate={'softmax':''})
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print(s,sf,f)
            result.append(qsf_f)
  return result

def dict_pos_neg(pos, neg, w):
  return dict(dict_same_weight(1.0*w/len(pos),pos), **dict_same_weight(-1.0*w/len(neg),neg))

def print_permutations(q):
#     for q in Q1s:
    W = q._pdf['W']
    print(q._descriptor)
    for i, (kmap, w) in enumerate(zip(q._keywords_map, W)):
        context = q._context_template.format_map(kmap)
        answer = q._answer_template.format_map(kmap)
#         sexisem_score = sexisem_classifier(context.strip('.') + ' ' +answer)
        print(f'{i}.',context ,'->', answer, w)
#     break


frequency_weights:SCALE = {
    'never':-4,
    'very rarely':-3,
    'seldom':-2,
    'rarely':-2,
    'frequently':2,
    'often':2,
    'very frequently':3,
    'always':4,    
}
    
    
binary_frequency_weights = {k: 1 if v > 0 else -1 for k, v in frequency_weights.items()}
rbinary_frequency_weights = {k: -1 if v > 0 else 1 for k, v in frequency_weights.items()}
frequency_pos = [k for k, v in frequency_weights.items() if v > 0]
frequency_neg = [k for k, v in frequency_weights.items() if v < 0]

In [4]:
# p = 'valhalla/distilbart-mnli-12-6'
p = "typeform/distilbert-base-uncased-mnli"
mnli = pipeline("zero-shot-classification",device=device, model=p)
mnli.model_identifier = p


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


# PHQ 2

In [5]:
from qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli import asi_bg_qmnli, asi_bi_qmnli, asi_bp_qmnli, asi_h_qmnli

In [6]:
for Q in asi_h_qmnli:
    Qs = split_question(Q,
                        index=Q.q_index,
                        scales=[Q.q_scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli).report()

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
2
Query time: 0.9372756481170654
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.050073776920423065
Internal consistency (silhouette, correlation) for unfiltered: 0.5499349236488342
Internal consistency (Calinski&Harabasz)  for unfiltered: 6.838287016655492
Internal consistency (Davies&Bouldin) for unfiltered: 0.6230203322544302


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qlatent\qabstract\qabstract.py:365: FutureWarning: The provided callable <function mean at 0x000001DE345D2D40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0002,0.0910,0.1302,0.0926,0.1599,0.1425,0.1864,0.1971
prejudiced,0.0022,0.0922,0.1218,0.0589,0.1699,0.1659,0.1887,0.2004
sexist,0.0568,0.0955,0.1150,0.1091,0.1458,0.1369,0.1736,0.1673
legitimate,0.0076,0.2342,0.1900,0.2329,0.1174,0.1566,0.0516,0.0098
reasonable,0.3000,0.1558,0.1160,0.1525,0.0701,0.0814,0.0554,0.0688
rightful,0.3088,0.1425,0.1145,0.1570,0.0842,0.0868,0.0561,0.0501


(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
4
Query time: 0.6499640941619873
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.105381477383465
Internal consistency (silhouette, correlation) for unfiltered: 0.8295049667358398
Internal consistency (Calinski&Harabasz)  for unfiltered: 80.00162251307546
Internal consistency (Davies&Bouldin) for unfiltered: 0.19556112058670908


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qlatent\qabstract\qabstract.py:365: FutureWarning: The provided callable <function mean at 0x000001DE345D2D40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


...

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qlatent\qabstract\qabstract.py:365: FutureWarning: The provided callable <function mean at 0x000001DE345D2D40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0001,0.0869,0.1225,0.0528,0.1824,0.1483,0.1853,0.2219
prejudiced,0.0001,0.0812,0.1130,0.0424,0.1850,0.1610,0.1885,0.2288
sexist,0.0001,0.0192,0.0189,0.0021,0.2360,0.2071,0.2425,0.2741
legitimate,0.1184,0.2320,0.2059,0.2616,0.0334,0.1045,0.0433,0.0008
reasonable,0.3245,0.1563,0.1456,0.2012,0.0484,0.0691,0.0481,0.0068
rightful,0.2566,0.1881,0.1569,0.2050,0.0601,0.0723,0.0443,0.0166


(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
21
Query time: 0.7565691471099854
Mean score unfiltered [-1.3333333333333333..1.3333333333333333]: -0.09992759535051239
Internal consistency (silhouette, correlation) for unfiltered: 0.7969951033592224
Internal consistency (Calinski&Harabasz)  for unfiltered: 62.399526199575554
Internal consistency (Davies&Bouldin) for unfiltered: 0.24026864714876553


index = ['index']
{'frequency', 'index'} {'frequency'} {'index'}
[]


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\qlatent\qabstract\qabstract.py:365: FutureWarning: The provided callable <function mean at 0x000001DE345D2D40> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df = pd.pivot_table(df, values='P', index=index, columns=[scale], aggfunc=np.mean)


frequency,never,very rarely,rarely,seldom,frequently,often,very frequently,always
index,,,,,,,,
improper,0.0001,0.0689,0.1099,0.1008,0.1846,0.1580,0.1824,0.1955
prejudiced,0.0000,0.0002,0.0430,0.0001,0.2333,0.2292,0.2404,0.2539
sexist,0.0001,0.0006,0.0367,0.0003,0.2426,0.2350,0.2471,0.2376
legitimate,0.2596,0.2763,0.2125,0.2502,0.0002,0.0010,0.0002,0.0001
reasonable,0.3272,0.2278,0.2003,0.2445,0.0000,0.0001,0.0000,0.0000
rightful,0.2468,0.2359,0.1818,0.2054,0.0314,0.0768,0.0215,0.0003
